# Representação de Trajetória
O objetivo deste notebook é apresentar os resultados obtidos de maneira visual.

In [1]:
import data_processing

data = data_processing.load_student_data()
disciplinas = data_processing.load_disciplinas()

aqui


In [2]:
sugestoes = []
with open('sugestoes.txt') as file:
    for line in file:
        nome = line.strip().split('\n')[0]
        if nome in disciplinas:
            disc = disciplinas[nome]
            sugestoes.append(disc)
            if disc.trilha == 'optativa' or disc.trilha == 'obrigatoria':
                print(f'{disc.nome} não está em nenhuma trilha?')
        else:
            print(f'{nome} não está em disciplinas')

In [3]:
for item in sugestoes:
    print(item.trilha)

Engenharia Biomedica
Optativas Isoladas
Desenvolvimento Baseado em Plataformas Programacao
Desenvolvimento Baseado em Plataformas Programacao
Sistemas Embarcados
Controle
IHC
IHC
Engenharia Biomedica
Desenvolvimento Baseado em Plataformas Programacao
Desenvolvimento Baseado em Plataformas Programacao
Otimizacao, Modelos Analiticos e de Simulacao
Optativas Isoladas
Banco De Dados
Sistemas Inteligentes
IHC
Banco De Dados
Redes De Computadores
Engenharia De Software
Sistemas Inteligentes
Otimizacao, Modelos Analiticos e de Simulacao
Optativas Isoladas
Redes De Computadores
Redes De Computadores
IHC
Sistemas Inteligentes
Engenharia De Software
IHC
Redes De Computadores
IHC
Fisica
Engenharia Biomedica
Fisica
Sistemas Inteligentes
Sistemas Inteligentes
Processamento Grafico
Algoritmos E Complexidade
Banco De Dados
Processamento Grafico
Banco De Dados
Processamento Grafico
Optativas Isoladas
Algoritmos E Complexidade
Processamento Grafico
Controle
Sistemas Inteligentes
Fisica
Fisica
Banco De 

In [5]:
from typing import List

def dar_sugestoes(CHS_desejada : int, sugestoes : List, ra_aluno : int):
    # Carga horária desejada
    CH_desejada = CHS_desejada * 15
